In [32]:
%matplotlib inline

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import twstock as ts
import time 
import threading
import queue
import json
from urllib.request import urlopen

# 獲取4月個股占大盤比重

In [33]:
table=pd.read_html("https://histock.tw/stock/taiexproportion.aspx")
smy=pd.concat([table[0],table[1]],axis=0)
smy.index=range(1,len(smy)+1)
smy=smy.drop('排行',axis=1)

In [34]:
smy['昨收']=np.nan
smy['現價']=np.nan

In [35]:
codelist=list(smy.loc[:,'代號'])

In [36]:
smy['市值佔大盤比重']=smy['市值佔大盤比重'].map(lambda x:float(x.strip('%'))) / 100

In [37]:
smy.head()

,代號,名稱,市值佔大盤比重,昨收,現價
1,2330,台積電,0.211929,NaN,NaN
2,2317,鴻海,0.042131,NaN,NaN
3,6505,台塑化,0.037738,NaN,NaN
4,2412,中華電,0.029486,NaN,NaN
5,1301,台塑,0.022606,NaN,NaN


## 從yahoo網頁獲取(即時)現價

In [7]:
def getnp(i):
    tab=pd.read_html(f"https://tw.stock.yahoo.com/q/q?s={i}",encoding = "big5")
    AD=tab[1][0][0].split()
    smy.loc[smy.loc[smy.代號==float(i)].index,'現價']=float(AD[14])

In [8]:
smy['現價']=np.nan
n=0
t0=time.clock()
while n<50:
    for i in codelist[n:n+10]:
        try:
            getnp(i)
        except:
            pass
    n+=10
    print('前',n,'間公司所需時間',time.clock()-t0,'秒')


前 10 間公司所需時間 5.26966128375146 秒
前 20 間公司所需時間 11.321755327001808 秒
前 30 間公司所需時間 16.749063316872075 秒
前 40 間公司所需時間 23.284519836386764 秒
前 50 間公司所需時間 29.28380318836629 秒


In [9]:
tab=pd.read_html("https://tw.stock.yahoo.com/q/q?s=2330",encoding = "big5")

In [10]:
AD=tab[1][0][0].split()
AD[14]

'261.5'

In [11]:
AD=tab[1][0][0].split()
AD[12]=AD[12][:-6]

In [12]:
col_name=AD[:11]

In [13]:
da_num=AD[12:23]

In [14]:
df=pd.DataFrame(columns=col_name)

In [30]:
df.loc['data']=da_num

In [31]:
df

,股票代號,時間,成交,買進,賣出,漲跌,張數,昨收,開盤,最高,最低
data,2330台積電,14:30,261.5,261.0,261.5,△4.5,"37,796",257.0,260.0,263.0,259.5


## 利用Market Information System的url與多線程  獲取個股昨日收盤價(昨收)

In [38]:
def run2(self):
    while self.qsize() > 0:
        i = self.get()
        if self.qsize()%100==0:
            print(f"剩下{self.qsize()}筆資料")
        try:
#            t0=time.clock()
            query_url = f"http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_{i}.tw&delay=0"
            data = json.loads(urlopen(query_url).read())
            smy.loc[smy.loc[smy.代號==float(i)].index,'昨收']=data['msgArray'][0]['y']
#            print(f'找{i}的昨收價花',time.clock()-t0,'秒')
        except:
            pass

In [39]:
my_queue = queue.Queue()
smy['現價']=np.nan
for i in codelist:
      my_queue.put(i)
t = []
t00=time.clock()
le=my_queue.qsize()
for i in range(5):
    t.append(threading.Thread(target = run2(my_queue)))
    t[i].start()
for i in range(5):    
    t[i].join()
tt=time.clock()-t00
print(f'總共花',tt,'秒')
print(f'平均花',tt/le,'秒')

剩下900筆資料
剩下800筆資料
剩下700筆資料
剩下600筆資料
剩下500筆資料
剩下400筆資料
剩下300筆資料
剩下200筆資料
剩下100筆資料
剩下0筆資料
總共花 68.59338607127756 秒
平均花 0.07554337673048189 秒


## 利用twstock.realtime.get('代碼') 與多線程  獲取個股現價

In [40]:
threading.active_count()

5

In [41]:
def run(self):
    while self.qsize() > 0:
        i = self.get()
        if self.qsize()%100==0:
            print(f"剩下{self.qsize()}筆資料")
        try:
#            t0=time.clock()
            stock=ts.realtime.get(f'{i}')
            stocknp=stock['realtime']['latest_trade_price']
            smy.loc[smy.loc[smy.代號==float(i)].index,'現價']=stocknp
#            print(f'找{i}的現價花',time.clock()-t0,'秒')
        except:
            pass

In [42]:
my_queue = queue.Queue()
smy['現價']=np.nan
for i in codelist:
      my_queue.put(i)
t = []
t00=time.clock()
le=my_queue.qsize()
for i in range(5):
    t.append(threading.Thread(target = run(my_queue)))
    t[i].start()
for i in range(5):    
    t[i].join()
tt=time.clock()-t00
print(f'總共花',tt,'秒')
print(f'平均花',tt/le,'秒')

剩下900筆資料
剩下800筆資料
剩下700筆資料
剩下600筆資料
剩下500筆資料
剩下400筆資料
剩下300筆資料
剩下200筆資料
剩下100筆資料
剩下0筆資料
總共花 151.94035283398807 秒
平均花 0.16733519034580185 秒


## 利用Market Information System的url 獲取加權指數昨收與現價

In [43]:
dapen=pd.DataFrame(columns=['yc','now','time'])

In [52]:
query_url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch=tse_t00.tw&json=1&delay=0&_=1554956220828"
data = json.loads(urlopen(query_url).read())
dic=data['msgArray'][0]
dapen.loc['加權指']=float(dic['y']),float(dic['z']),dic['t']
dapen

KeyError: 'z'

# 計算即時的加權指數與台指期進行比較

In [45]:
rate_sum=0
for i in range(1,len(smy.index)+1):
    rate=smy.loc[i,'市值佔大盤比重']
    rate_sum=rate_sum+rate
rate_sum

1.0519509999999987

In [46]:
nr=smy['市值佔大盤比重'].copy()
nr=nr/rate_sum

In [47]:
findex=0
smy=smy.dropna()
smy.index=range(1,len(smy.index)+1)
for i in range(1,len(smy.index)+1):
    findex+=dapen.loc['加權指','yc']*(1+(float(smy.loc[i,'現價'])-float(smy.loc[i,'昨收']))/float(smy.loc[i,'昨收']))*nr[i]

In [48]:
findex-70

-70

In [49]:
dg=pd.read_csv("http://info512ah.taifex.com.tw/Future/ImgChartDetail.ashx?type=1&contract=MTX059",encoding='big5',
               sep='delimiter',engine='python')
dg.loc[[4,11]]

,小臺指期059
4,"成交,11014.0000,漲跌,22"
11,"時間,045959"


In [51]:
smy

,代號,名稱,市值佔大盤比重,昨收,現價


In [53]:
encodeURIComponent()

NameError: name 'encodeURIComponent' is not defined